# 1、字符串输出解析器 StrOutputParser

In [1]:
# 1、获取大模型
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser, XMLOutputParser

import os
import dotenv
from langchain_core.utils import pre_init
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

chat_model = ChatOpenAI(model="Qwen/Qwen3-8B")

# 2、调用大模型
response = chat_model.invoke("什么是大语言模型？")
# print(type(response))   #AIMessage

#3、如何获取一个字符串的输出结果呢？
# 方式1：自己调用输出结果的content
# print(response.content)

# 方式2：使用StrOutputParser
parser = StrOutputParser()
str_response = parser.invoke(response)
print(type(str_response))  #<class 'str'>
print(str_response)


<class 'str'>
大语言模型（Large Language Models，简称LLMs）是一种基于深度学习技术的自然语言处理（NLP）模型，其特点是拥有**极大规模的参数量**，通常达到数亿乃至数千亿个参数，并且在**海量文本数据上进行训练**，从而具备强大的语言理解和生成能力。

### 大语言模型的几个关键特点：

1. **参数量巨大**：
   - LLMs一般由数百万到数千亿个参数组成。
   - 更多的参数意味着模型可以捕捉更复杂的语言模式和语义信息。

2. **大规模预训练**：
   - 模型通常在一个非常大的文本语料库上进行预训练，比如互联网上的文本、书籍、文章等。
   - 预训练阶段使模型掌握广泛的语言知识，包括语法、语义、常识等。

3. **多任务能力**：
   - 大语言模型可以执行多种语言相关的任务，如文本生成、问答、翻译、摘要、代码生成、逻辑推理、写作、编程等。

4. **强大的上下文理解能力**：
   - 它们可以理解自然语言中的上下文关系，并在对话或文章中保持连贯性。
   - 支持长文本输入与输出，能处理复杂的对话和多轮交互。

5. **无需任务特定训练**：
   - 通过**微调（fine-tuning）**或**提示（prompting）**，LLMs可以在少量数据上快速适应各种具体任务。

6. **开源与闭源**：
   - 一些LLMs是**开源**的，如HuggingFace的GPT-NeoX、Facebook的BART、Google的T5等。
   - 也有一些是通过闭源方式进行部署的，如OpenAI的GPT系列、百度的文心一言、阿里云的通义千问等。

---

### 举例说明大语言模型（LLMs）：

- **GPT系列**：由OpenAI开发，包括GPT-1、GPT-2、GPT-3、GPT-3.5、GPT-4等，是当前最著名的LLMs之一。
- **BERT系列**：由Google开发，用于理解上下文的双向Transformer模型。
- **T5**：Google的多任务模型，可以完成多个NLP任务。
- **LLaMA**、**Llama2**：由Meta开发，开源的系列模型。
- **Qwen（通义千问）**、**ERNIE Bot**、**通义万相**：阿里云研发的一系列模型。


# 2、JsonOutputParser : Json输出解析器

方式1：

In [2]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOpenAI(model="Qwen/Qwen3-8B")

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个靠谱的{role}"),
    ("human", "{question}")
])

# 正确的：
prompt = chat_prompt_template.invoke(
    input={"role": "人工智能专家", "question": "人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式的数据"})

# 错误的：
# prompt = chat_prompt_template.invoke(input={"role":"人工智能专家","question":"人工智能用英文怎么说？"})

response = chat_model.invoke(prompt)
# print(response.content)

# 获取一个JsonOutputParser的实例
parser = JsonOutputParser()

json_result = parser.invoke(response)
print(json_result)

{'q': '人工智能用英文怎么说？', 'a': 'Artificial Intelligence'}


方式2：

举例1：

In [3]:
parser = JsonOutputParser()

print(parser.get_format_instructions())

Return a JSON object.


举例2：

In [4]:
# 引入依赖包
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# 初始化语言模型
chat_model = ChatOpenAI(model="Qwen/Qwen3-8B")

joke_query = "告诉我一个笑话。"

# 定义Json解析器
parser = JsonOutputParser()

#以PromptTemplate为例
prompt_template = PromptTemplate.from_template(
    template="回答用户的查询\n 满足的格式为{format_instructions}\n 问题为{question}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt = prompt_template.invoke(input={"question": joke_query})
response = chat_model.invoke(prompt)
print(response)

json_result = parser.invoke(response)
print(json_result)

content='{\n  "笑话": "为什么数学书总是很忧郁？因为它有太多的问题。"\n}' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 35, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '019bd04786930b7d81c28ac0f959533b', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019bd047-8563-7052-a29d-75edc0dbc73c-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 35, 'output_tokens': 21, 'total_tokens': 56, 'input_token_details': {}, 'output_token_details': {'reasoning': 0}}
{'笑话': '为什么数学书总是很忧郁？因为它有太多的问题。'}


知识的拓展： |

In [5]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOpenAI(model="Qwen/Qwen3-8B")

chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个靠谱的{role}"),
    ("human", "{question}")
])

# 获取一个JsonOutputParser的实例
parser = JsonOutputParser()

# 写法1：
# prompt = chat_prompt_template.invoke(input={"role":"人工智能专家","question":"人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式的数据"})
#
# response = chat_model.invoke(prompt)
#
# json_result = parser.invoke(response)
# print(json_result)


# 写法2：
# 提示词 》》 模型 》》 结果解析器
chain = chat_prompt_template | chat_model | parser
json_result1 = chain.invoke(
    input={"role": "人工智能专家", "question": "人工智能用英文怎么说？问题用q表示，答案用a表示，返回一个JSON格式的数据"})
print(json_result1)

{'q': '人工智能用英文怎么说？', 'a': 'Artificial Intelligence'}


针对于举例2

In [6]:
# 引入依赖包
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# 初始化语言模型
chat_model = ChatOpenAI(model="Qwen/Qwen3-8B")

joke_query = "告诉我一个笑话。"

# 定义Json解析器
parser = JsonOutputParser()

#以PromptTemplate为例
prompt_template = PromptTemplate.from_template(
    template="回答用户的查询\n 满足的格式为{format_instructions}\n 问题为{question}\n",
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
# 写法1：
# prompt = prompt_template.invoke(input={"question":joke_query})
# response = chat_model.invoke(prompt)
# json_result = parser.invoke(response)

chain = prompt_template | chat_model | parser
json_result = chain.invoke(input={"question": joke_query})
print(json_result)

{'joke': '为什么数学书总是很忧郁？', 'answer': '因为它有太多的问题。'}


# 3、XMLOutputParser XML输出解析器的使用

举例1：自己在提示词模板中写明使用XML格式

In [7]:
chat_model = ChatOpenAI(model="Qwen/Qwen3-8B")

actor_query = "周星驰的简短电影记录"
response = chat_model.invoke(f"请生成{actor_query}，将影片附在<movie></movie>标签中")

print(type(response))
print(response.content)

<class 'langchain_core.messages.ai.AIMessage'>
<movie>
  <title>喜剧之王</title>
  <year>1999</year>
  <director>周星驰</director>
  <genre>喜剧 / 爱情</genre>
  <summary>
    《喜剧之王》讲述了小人物尹天仇在追求梦想路上的经历。他在电影圈中努力打拼，与女演员柳飘飘相恋，但面对现实的打击和理想与现实的冲突，最终在一次演出中获得认可，实现自我价值。影片以真挚的情感和幽默的风格，展现了奋斗与爱的主题。
  </summary>
</movie>

<movie>
  <title>大话西游</title>
  <year>1995</year>
  <director>周星驰</director>
  <genre>科幻 / 喜剧</genre>
  <summary>
    《大话西游》是周星驰最具影响力的电影之一，将经典西游记故事与现代爱情元素结合，讲述至尊宝与紫霞仙子跨越时空的爱情。影片独特的叙事结构与夸张的喜剧效果使其成为华语影史经典。
  </summary>
</movie>

<movie>
  <title>少林足球</title>
  <year>2001</year>
  <director>周星驰</director>
  <genre>喜剧 / 励志</genre>
  <summary>
    《少林足球》是周星驰自编自导自演的电影，讲述了足球队通过少林功夫训练来提升球技，最终在比赛中逆袭的故事。影片充满荒诞幽默与热血元素，是周星驰风格的代表作之一。
  </summary>
</movie>


举例2：

In [8]:
from langchain_core.output_parsers.xml import XMLOutputParser

parser = XMLOutputParser()
print(parser.get_format_instructions())

The output should be formatted as a XML file.
1. Output should conform to the tags below.
2. If tags are not given, make them on your own.
3. Remember to always open and close all the tags.

As an example, for the tags ["foo", "bar", "baz"]:
1. String "<foo>
   <bar>
      <baz></baz>
   </bar>
</foo>" is a well-formatted instance of the schema.
2. String "<foo>
   <bar>
   </foo>" is a badly-formatted instance.
3. String "<foo>
   <tag>
   </tag>
</foo>" is a badly-formatted instance.

Here are the output tags:
```
None
```


使用parser.get_format_instructions()结构实现：

In [10]:
# 1.导入相关包
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 2. 初始化语言模型
chat_model = ChatOpenAI(model="Qwen/Qwen3-8B")

# 3.测试模型的xml解析效果
actor_query = "生成汤姆·汉克斯的简短电影记录,使用中文回复"

# 4.定义XMLOutputParser对象
parser = XMLOutputParser()

# 5. 生成提示词模板
prompt_template1 = PromptTemplate.from_template(
    template="用户的问题：{query}\n使用的格式：{format_instructions}"
)

prompt_template2 = prompt_template1.partial(format_instructions=parser.get_format_instructions())


response = chat_model.invoke(prompt_template2.invoke(input={"query": actor_query}))
print(response.content)


```xml
<电影记录>
   <演员>
      <姓名>汤姆·汉克斯</姓名>
      <国籍>美国</国籍>
      <职业>演员、导演、制片人</职业>
   </演员>
   <代表作品>
      <电影>
         <名称>阿甘正传</名称>
         <年份>1994</年份>
         <角色>阿甘</角色>
         <奖项>奥斯卡最佳男主角奖</奖项>
      </电影>
      <电影>
         <名称>绿里奇迹</名称>
         <年份>1999</年份>
         <角色>保罗·L·奥布莱恩</角色>
         <奖项>奥斯卡最佳男主角奖</奖项>
      </电影>
      <电影>
         <名称>荒岛余生</名称>
         <年份>2000</年份>
         <角色>查克·诺兰</角色>
         <奖项>金球奖最佳男主角奖</奖项>
      </电影>
      <电影>
         <名称>拯救大兵瑞恩</名称>
         <年份>1998</年份>
         <角色>队长米勒</角色>
         <奖项>奥斯卡最佳男配角奖</奖项>
      </电影>
      <电影>
         <名称>天堂电影院</名称>
         <年份>1988</年份>
         <角色>阿尔弗雷多·“弗雷多”·托雷</角色>
         <奖项>威尼斯电影节银狮奖</奖项>
      </电影>
   </代表作品>
   <荣誉>
      <奖项>奥斯卡金像奖最佳男主角（2次）</奖项>
      <奖项>奥斯卡金像奖最佳男配角（1次）</奖项>
      <奖项>金球奖最佳男主角（2次）</奖项>
      <奖项>金球奖最佳男配角（1次）</奖项>
   </荣誉>
</电影记录>
```


In [ ]:
xml_result = parser.invoke(response)
print(xml_result)

# 4、列表解析器 CommaSeparatedListOutputParser

举例1：

In [11]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()

# 返回一些指令或模板，这些指令告诉系统如何解析或格式化输出数据
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

messages = "大象,猩猩,狮子"
result = output_parser.parse(messages)
print(result)
print(type(result))

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
['大象', '猩猩', '狮子']
<class 'list'>


举例2：

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser

# 初始化语言模型
chat_model = ChatOpenAI(model="gpt-4o-mini")

# 创建解析器
output_parser = CommaSeparatedListOutputParser()

# 创建LangChain提示模板
chat_prompt = PromptTemplate.from_template(
    "生成5个关于{text}的列表.\n\n{format_instructions}",
    partial_variables={
    "format_instructions": output_parser.get_format_instructions()
    })

# 提示模板与输出解析器传递输出
# chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions())

# 将提示和模型合并以进行调用
chain = chat_prompt | chat_model | output_parser
res = chain.invoke({"text": "电影"})
print(res)
print(type(res))

# 5、日期解析器 DatetimeOutputParser

举例1：

In [ ]:
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser()

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

举例2：


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import DatetimeOutputParser

chat_model = ChatOpenAI(model="gpt-4o-mini")


chat_prompt = ChatPromptTemplate.from_messages([
    ("system","{format_instructions}"),
    ("human", "{request}")
])

output_parser = DatetimeOutputParser()

# 方式1：
# model_request = chat_prompt.format_messages(
#     request="中华人民共和国是什么时候成立的",
#     format_instructions=output_parser.get_format_instructions()
# )

# response = chat_model.invoke(model_request)
# result = output_parser.invoke(response)
# print(result)
# print(type(result))

# 方式2：
chain = chat_prompt | chat_model | output_parser
resp = chain.invoke({"request":"中华人民共和国是什么时候成立的",
                     "format_instructions":output_parser.get_format_instructions()})
print(resp)
print(type(resp))